## Install Ktor

- ktor-client-core
- ktor-client-cio

In [1]:
USE {
    val ktorVersion = "3.1.0"

    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("io.ktor:ktor-client-core:$ktorVersion")
        implementation("io.ktor:ktor-client-cio:$ktorVersion")
    }
}

In [2]:
%use ktor-client
%use dataframe

In [3]:
import io.ktor.client.*
import io.ktor.client.engine.cio.*
import io.ktor.client.request.*
import io.ktor.client.statement.*
import kotlinx.coroutines.*
import java.time.*
import java.math.BigDecimal
import java.time.format.DateTimeFormatter

val client = HttpClient(CIO)

@Serializable
data class ApiResponse(
    val code: String,
    val msg: String,
    val requestTime: Long,
    val data: List<List<String>>,
)

data class CandleRecord(
    val timestamp: Long,
    val open: BigDecimal,
    val high: BigDecimal,
    val low: BigDecimal,
    val close: BigDecimal,
    val volume: BigDecimal,
    val quote_currency_volume: BigDecimal,
)

suspend fun fetchCandles(
    symbol: String = "BTCUSDT",
    granularity: String = "1m",
    startDate: String = LocalDate.now().minusDays(1).format(DateTimeFormatter.ISO_DATE),
    endDate: String = LocalDate.now().format(DateTimeFormatter.ISO_DATE),
): List<CandleRecord> {
    require(startDate < endDate) { "startDate must be earlier than endDate" }
    require(granularity in setOf("1m", "3m", "5m", "15m", "30m", "1H", "4h", "1d", "1w", "1M")) {
        "granularity must be one of 1m, 5m, 15m, 30m, 1H, 4h, 1d, 1w, 1M"
    }
    require(symbol.isNotBlank()) { "symbol must not be blank" }

    // granularity 문자열에 따른 밀리초 단위 시간 간격 계산
    val granularityMillis = when (granularity) {
        "1m" -> 60_000L
        "3m" -> 180_000L
        "5m" -> 300_000L
        "15m" -> 900_000L
        "30m" -> 1_800_000L
        "1H", "1h" -> 3_600_000L
        "4h", "4H" -> 14_400_000L
        "1d" -> 86_400_000L
        "1w" -> 604_800_000L
        "1M" -> 2_629_800_000L  // 대략적인 값
        else -> throw IllegalArgumentException("Unsupported granularity: $granularity")
    }

    // startDate와 endDate를 밀리초로 변환
    val startMillis: Long = LocalDate.parse(startDate)
        .atStartOfDay().toInstant(ZoneOffset.UTC).toEpochMilli()
    // endDate는 포함되어야 하므로, 다음 날 00:00으로 설정
    val globalEndMillis: Long = LocalDate.parse(endDate)
        .plusDays(1).atStartOfDay().toInstant(ZoneOffset.UTC).toEpochMilli()

    suspend fun fetchCandlesChunk(
        symbol: String,
        granularity: String,
        startTime: Long,
        endTime: Long,
    ): List<CandleRecord> {
        // URL을 한 줄로 구성(줄바꿈 제거)
        val url = """
          https://api.bitget.com/api/v2/mix/market/history-candles?symbol=$symbol&granularity=$granularity&limit=200&productType=usdt-futures&startTime=$startTime&endTime=$endTime
        """.trimIndent().replace("\n", "")

        return client.get(url)
            .readBytes()
            .decodeToString()
            .let { Json.decodeFromString<ApiResponse>(it) }
            .data.map { row ->
                CandleRecord(
                    timestamp = row[0].toLong(),
                    open = row[1].toBigDecimal(),
                    high = row[2].toBigDecimal(),
                    low = row[3].toBigDecimal(),
                    close = row[4].toBigDecimal(),
                    volume = row[5].toBigDecimal(),
                    quote_currency_volume = row[6].toBigDecimal(),
                )
            }
    }

    val results = mutableListOf<CandleRecord>()
    var currentStart = startMillis

    // 한 번에 최대 200개의 캔들에 해당하는 범위를 사용하여 페이지네이션 진행
    while (currentStart < globalEndMillis) {
        val currentEnd = minOf(currentStart + 200 * granularityMillis, globalEndMillis)
        val chunk = fetchCandlesChunk(
            symbol = symbol,
            granularity = granularity,
            startTime = currentStart,
            endTime = currentEnd
        )

        if (chunk.isEmpty()) break

        println("""
            Fetching candles for $symbol
            from ${Instant.ofEpochMilli(currentStart).atZone(ZoneOffset.UTC).toLocalDateTime()}
            to ${Instant.ofEpochMilli(currentEnd).atZone(ZoneOffset.UTC).toLocalDateTime()},
            ${chunk.size} records
        """.trimIndent())

        results.addAll(chunk)

        // 업데이트: 다음 호출은 마지막 캔들의 timestamp + granularityMillis부터
        currentStart = chunk.last().timestamp + granularityMillis + 1
        if (chunk.size < 200) break  // 더 이상 데이터가 없는 경우 종료
    }

    return results
}

In [5]:
import java.time.*
import java.time.Instant

val symbol = "XRPUSDT"
val startDate = "2024-12-01"
val endDate = "2025-02-20"
val granularity = "15m"

val df = runBlocking {
    fetchCandles(
        symbol = symbol,
        granularity = granularity,
        startDate = startDate,
        endDate = endDate
    )
        .toDataFrame<CandleRecord>()
        .apply {
            distinctBy { "timestamp"<Long>() }
            sortBy { "timestamp"<Long>() }
        }
        .let {
            it.insert("time") {
                Instant.ofEpochMilli("timestamp"<Long>())
                    .atZone(ZoneOffset.UTC).toLocalDateTime()
            }.after("timestamp")
        }
}

Fetching candles for XRPUSDT
from 2024-12-01T00:00
to 2024-12-03T02:00,
200 records
Fetching candles for XRPUSDT
from 2024-12-03T02:00:00.001
to 2024-12-05T04:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-05T04:00:00.001
to 2024-12-07T06:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-07T06:00:00.001
to 2024-12-09T08:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-09T08:00:00.001
to 2024-12-11T10:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-11T10:00:00.001
to 2024-12-13T12:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-13T12:00:00.001
to 2024-12-15T14:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-15T14:00:00.001
to 2024-12-17T16:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-17T16:00:00.001
to 2024-12-19T18:00:00.001,
200 records
Fetching candles for XRPUSDT
from 2024-12-19T18:00:00.001
to 2024-12-21T20:00:00.001,
200 records
Fetching candles for XRPUSDT
from 

In [6]:
df.writeCSV("${symbol}_${granularity}_candles_${startDate}_$endDate.csv")